# IBMD Data Analysis
##### Planned Action

1. Define the business problem
   ¬¬ Come up with a possible business questions
2.  Import data
   ¬¬ Write out a short metedata description for each table
   ¬¬ Identify possible tables from the imported data that will answer the business question
3.  Inspect the chosen dataset
   ¬¬ Outline possible problems that could occur with the data
   ¬¬ Profiling the data 
4.  Transform the chosen dataset
   ¬¬ Cleaning the data
   ¬¬ Enrich the data if needed to meet business needs
5.  Verify the transformed dataset
6.  Build the data pipeline - ELT or ETL



In [ ]:
import pandas as pd
from src.transformation import transform_data

In [ ]:
def check_null_values(movie_data, needed_table):
    for table, data in movie_data.items():
        if table in needed_table:
            print(table)
            null_count = movie_data[table].isna().sum()
            print(null_count)

def remove_dollar_signs_comma(movie_data, COLUMNS_TO_TRANSFORM ):
    for table_name, columns in COLUMNS_TO_TRANSFORM .items():
        for column in columns:
            print(column)
            if movie_data[table_name][column].dtype == 'object':
                movie_data[table_name][column] = movie_data[table_name][column].str.replace('$', '').str.replace(',', '')
    return movie_data

def change_data_type(movie_data, COLUMNS_TO_TRANSFORM):
    for table_name, columns in COLUMNS_TO_TRANSFORM .items():
        for column in columns:
            print(column)
            movie_data[table_name][column] = pd.to_numeric(movie_data[table_name][column], errors='coerce')
    return movie_data

In [1]:


# Function to import data from the given URLs
def import_data(tables):
    movies = {}
    for url, table_name in tables.items():
        if url.endswith('.csv'):
            movies[table_name] = pd.read_csv(url, engine="python")
        elif url.endswith('.tsv'):
            movies[table_name] = pd.read_csv(url, delimiter='\t', engine="python")
    return movies

# URLs of the data tables
TABLES = {
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/MovieLens_movies.csv": "movies_Id",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/IMDb%20BoxOfficeMojo%20-%20Brands%20(US%20%26%20Canada).tsv": "brands_US_and_Canada",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/IMDb%20BoxOfficeMojo%20-%20Brand_%20Marvel%20Comics.tsv": "brand_marvel_comics",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/The%20Numbers%20-%20Domestic%20Box%20Office%20Daily%20-%20The%20Avengers.tsv": "Domestic_Box_Office_Daily_The_Avengers",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/The%20Numbers%20-%20Domestic%20Box%20Office%20-%20Franchises.tsv": "Domestic_Box_Office_Franchises",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/The%20Numbers%20-%20Domestic%20Box%20Office%20-%20Franchises%20-%20Marvel%20Cinematic%20Universe.tsv": "Domestic_Box_Office_Franchises_Marvel_Cinematic",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/World%20Wide%20Box%20Office%20All%20Time%20Top%201000.tsv": "World_Wide_Box_Office_All_Time_Top_1000",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/IMDb%20BoxOfficeMojo%20-%20Franchises%20(US%20%26%20Canada).tsv": "Franchises_us_and_Canada",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/IMDb%20BoxOfficeMojo%20-%20Franchise_%20top20.tsv": "top_20_for_each_Franchise",
    "https://raw.githubusercontent.com/mansik95/IMDB-Analysis/master/Data/MovieLens_tags.csv": "tags"
    }

# Importing data
movie_data = import_data(TABLES)

##### Handling Missing Values from the Movie_dataset
For the purpose of this project we need only four tables for this data analysis.
##### Needed tables to answer business
1. DOMESTIC_BOX_OFFICE_FRANCHISES_MARVEL_CINEMATIC
2. DOMESTIC_BOX_OFFICE_FRANCHISES
3. WORLD_WIDE_BOX_OFFICE_ALL_TIME_TOP_1000
4. TOP_20_FOR_EACH_FRANCHISE

In [5]:
needed_table = ["tags","Domestic_Box_Office_Franchises_Marvel_Cinematic","movies_Id","brands_US_and_Canada","Domestic_Box_Office_Franchises","World_Wide_Box_Office_All_Time_Top_1000","top_20_for_each_Franchise"]

check_null_values(movie_data, needed_table)

movies_Id
movieId    0
title      0
genres     0
dtype: int64
brands_US_and_Canada
Brand             0
Total_Gross       0
Releases          0
No_1_Release      0
Lifetime_Gross    0
dtype: int64
Domestic_Box_Office_Franchises
Franchise                    0
No_of_Movies                 0
Domestic_Box_Office          0
Infl_Adj_Dom_Box_Office      0
Worldwide_Box_Office         0
First_Year                   0
Last_Year                    0
No_of_Years                135
dtype: int64
Domestic_Box_Office_Franchises_Marvel_Cinematic
Release_Date            0
Title                   0
Production_Budget       5
Opening_Weekend         5
Domestic_Box_Office     5
Worldwide_Box_Office    5
dtype: int64
World_Wide_Box_Office_All_Time_Top_1000
Rank                      0
Title                     0
WorldwideLifetimeGross    0
Domesti LifetimeGross     0
Domestic%                 4
ForeignLifetimeGross      0
Foreign%                  0
Year                      0
dtype: int64
top_20_for_each_Fr

##### Observation from finding missing values

##### **Numbers of Missing values :**
1. World_Wide_Box_Office_All_Time_Top_1000
¬¬¬ Domestic%                 4

2. tags
¬¬¬ tag          16

3. Domestic_Box_Office_Franchises_Marvel_Cinematic
¬¬¬ Production_Budget       5
¬¬¬ Opening_Weekend         5
¬¬¬ Domestic_Box_Office     5
¬¬¬ Worldwide_Box_Office    5

4. Domestic_Box_Office_Franchises
No_of_Years                135

##### **Handling Missing Values:**
--From the veiwing the "No_of_years" column in the "Domestic_Box_Office_Franchises" Table  can see that instead of saving zero, the column was saving it as NaN.

In [ ]:
movie_data["Domestic_Box_Office_Franchises"] = movie_data["Domestic_Box_Office_Franchises"].fillna(0)
movie_data["tags"] = movie_data["tags"].dropna()
movie_data["Domestic_Box_Office_Franchises_Marvel_Cinematic"] =  movie_data["Domestic_Box_Office_Franchises_Marvel_Cinematic"].dropna()
movie_data["World_Wide_Box_Office_All_Time_Top_1000"] = movie_data["World_Wide_Box_Office_All_Time_Top_1000"].dropna()

# Let verify if we were able to remove all the null values
check_null_values(movie_data, needed_table)

In [ ]:
# Ensuring all the needed tables are in their right formats

def check_data_type(movie_data, needed_table):
    for table, data in movie_data.items():
        if table in needed_table:
            print(table)
            print((movie_data[table]).info())

check_data_type(movie_data, needed_table)

------ list of tables that require transformation 

##### DOMESTIC_BOX_OFFICE_FRANCHISES
1.   Domestic_Box_Office      867 non-null    object ---- remove the dollar sign and convert type to int
2.   Infl_Adj_Dom_Box_Office  867 non-null    object to int ---- remove the dollar sign and convert type to int
3.   Worldwide_Box_Office     867 non-null    object to int ---- remove the dollar sign and convert type to int

##### DOMESTIC_BOX_OFFICE_FRANCHISES_MARVEL_CINEMATIC
1.   Production_Budget     23 non-null     object to int ---- remove the dollar sign and convert type to int
2.   Opening_Weekend       23 non-null     object to int ---- remove the dollar sign and convert type to int
3.   Domestic_Box_Office   23 non-null     object to int ---- remove the dollar sign and convert type to int
4.   Worldwide_Box_Office  23 non-null     object to int ---- remove the dollar sign and convert type to int

##### TOP_20_FOR_EACH_FRANCHISE
1.   Lifetime_Gross  281 non-null    object to int ---- remove the dollar sign and convert type to int
2.   Max_Theaters    281 non-null    object to int ---- remove the comma sign and convert type to int
3.   Opening_Gross   281 non-null    object to int ---- remove the dollar sign and convert type to int
4.   Open_Theaters   281 non-null    object to int ---- remove the comma sign and convert type to int


In [43]:
# Columns to remove the dollar sign from

COLUMNS_TO_TRANSFORM = {
    'Domestic_Box_Office_Franchises': ['Domestic_Box_Office', 'Infl_Adj_Dom_Box_Office', 'Worldwide_Box_Office'],
    'Domestic_Box_Office_Franchises_Marvel_Cinematic': ['Production_Budget', 'Opening_Weekend', 'Domestic_Box_Office', 'Worldwide_Box_Office'],
    'top_20_for_each_Franchise': ['Lifetime_Gross','Opening_Gross','Max_Theaters']
}

movie_data = remove_dollar_signs_comma(movie_data, COLUMNS_TO_TRANSFORM)
change_data_type(movie_data, COLUMNS_TO_TRANSFORM)

Domestic_Box_Office
Infl_Adj_Dom_Box_Office
Worldwide_Box_Office
Production_Budget
Opening_Weekend
Domestic_Box_Office
Worldwide_Box_Office
Lifetime_Gross
Opening_Gross
Max_Theaters
Domestic_Box_Office
Infl_Adj_Dom_Box_Office
Worldwide_Box_Office
Production_Budget
Opening_Weekend
Domestic_Box_Office
Worldwide_Box_Office
Lifetime_Gross
Opening_Gross
Max_Theaters


{'movies_Id':        movieId                               title  \
 0            1                    Toy Story (1995)   
 1            2                      Jumanji (1995)   
 2            3             Grumpier Old Men (1995)   
 3            4            Waiting to Exhale (1995)   
 4            5  Father of the Bride Part II (1995)   
 ...        ...                                 ...   
 62418   209157                           We (2018)   
 62419   209159           Window of the Soul (2001)   
 62420   209163                    Bad Poems (2018)   
 62421   209169                 A Girl Thing (2001)   
 62422   209171      Women of Devil's Island (1962)   
 
                                             genres  
 0      Adventure|Animation|Children|Comedy|Fantasy  
 1                       Adventure|Children|Fantasy  
 2                                   Comedy|Romance  
 3                             Comedy|Drama|Romance  
 4                                           Comedy  
 

In [ ]:
transformed_data = transform_data(movie_data)